[参考](https://www.kaggle.com/nitinvijay23/predict-the-crime-category-knn-logistic)。

In [1]:
import pandas as pd
import numpy as np

In [4]:
%%time

train_data = pd.read_csv('../input/train.csv', parse_dates=['Dates'])
test_data = pd.read_csv('../input/test.csv', parse_dates=['Dates'], index_col='Id')

CPU times: user 3.54 s, sys: 275 ms, total: 3.81 s
Wall time: 3.82 s


In [5]:
target = train_data["Category"].unique()
print(target)

['WARRANTS' 'OTHER OFFENSES' 'LARCENY/THEFT' 'VEHICLE THEFT' 'VANDALISM'
 'NON-CRIMINAL' 'ROBBERY' 'ASSAULT' 'WEAPON LAWS' 'BURGLARY'
 'SUSPICIOUS OCC' 'DRUNKENNESS' 'FORGERY/COUNTERFEITING' 'DRUG/NARCOTIC'
 'STOLEN PROPERTY' 'SECONDARY CODES' 'TRESPASS' 'MISSING PERSON' 'FRAUD'
 'KIDNAPPING' 'RUNAWAY' 'DRIVING UNDER THE INFLUENCE'
 'SEX OFFENSES FORCIBLE' 'PROSTITUTION' 'DISORDERLY CONDUCT' 'ARSON'
 'FAMILY OFFENSES' 'LIQUOR LAWS' 'BRIBERY' 'EMBEZZLEMENT' 'SUICIDE'
 'LOITERING' 'SEX OFFENSES NON FORCIBLE' 'EXTORTION' 'GAMBLING'
 'BAD CHECKS' 'TREA' 'RECOVERED VEHICLE' 'PORNOGRAPHY/OBSCENE MAT']


In [9]:
data_dict = {}
count = 1
for data in target:
    data_dict[data] = count
    count += 1
print(data_dict)

{'WARRANTS': 1, 'OTHER OFFENSES': 2, 'LARCENY/THEFT': 3, 'VEHICLE THEFT': 4, 'VANDALISM': 5, 'NON-CRIMINAL': 6, 'ROBBERY': 7, 'ASSAULT': 8, 'WEAPON LAWS': 9, 'BURGLARY': 10, 'SUSPICIOUS OCC': 11, 'DRUNKENNESS': 12, 'FORGERY/COUNTERFEITING': 13, 'DRUG/NARCOTIC': 14, 'STOLEN PROPERTY': 15, 'SECONDARY CODES': 16, 'TRESPASS': 17, 'MISSING PERSON': 18, 'FRAUD': 19, 'KIDNAPPING': 20, 'RUNAWAY': 21, 'DRIVING UNDER THE INFLUENCE': 22, 'SEX OFFENSES FORCIBLE': 23, 'PROSTITUTION': 24, 'DISORDERLY CONDUCT': 25, 'ARSON': 26, 'FAMILY OFFENSES': 27, 'LIQUOR LAWS': 28, 'BRIBERY': 29, 'EMBEZZLEMENT': 30, 'SUICIDE': 31, 'LOITERING': 32, 'SEX OFFENSES NON FORCIBLE': 33, 'EXTORTION': 34, 'GAMBLING': 35, 'BAD CHECKS': 36, 'TREA': 37, 'RECOVERED VEHICLE': 38, 'PORNOGRAPHY/OBSCENE MAT': 39}


In [10]:
train_data["Category"] = train_data["Category"].replace(data_dict)

In [12]:
# Replacing the day of weeks
data_week_dict = {
    "Monday": 1,
    "Tuesday": 2,
    "Wednesday": 3,
    "Thursday": 4,
    "Friday": 5,
    "Saturday": 6,
    "Sunday": 7
}
train_data["DayOfWeek"] = train_data["DayOfWeek"].replace(data_week_dict)
test_data["DayOfWeek"] = test_data["DayOfWeek"].replace(data_week_dict)

In [14]:
# District
district = train_data["PdDistrict"].unique()
district

array(['NORTHERN', 'PARK', 'INGLESIDE', 'BAYVIEW', 'RICHMOND', 'CENTRAL',
       'TARAVAL', 'TENDERLOIN', 'MISSION', 'SOUTHERN'], dtype=object)

In [15]:
data_dict_district = {}
count = 1
for data in district:
    data_dict_district[data] = count
    count += 1
train_data["PdDistrict"] = train_data["PdDistrict"].replace(data_dict_district)
test_data["PdDistrict"] = test_data["PdDistrict"].replace(data_dict_district)

In [16]:
columns_train = train_data.columns
print(columns_train)
columns_test = test_data.columns
print(columns_test)

Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y'],
      dtype='object')
Index(['Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y'], dtype='object')


In [17]:
cols = columns_train.drop("Resolution")

In [21]:
train_data_new = train_data[cols]

In [22]:
corr = train_data_new.corr()
print(corr["Category"])

Category      1.000000
DayOfWeek    -0.016263
PdDistrict    0.007643
X             0.000147
Y            -0.005303
Name: Category, dtype: float64


There is no strong correlation of category with any numeric value.

In [23]:
skew = train_data_new.skew()
print(skew)

Category        1.662607
DayOfWeek      -0.005572
PdDistrict     -0.232137
X              18.685494
Y             113.984988
dtype: float64


In [24]:
features = ["DayOfWeek", "PdDistrict", "X", "Y"]
X_train = train_data[features]
y_train = train_data["Category"]
X_test = test_data[features]

In [26]:
X_train.head()

,DayOfWeek,PdDistrict,X,Y
0,3,1,-122.425892,37.774599
1,3,1,-122.425892,37.774599
2,3,1,-122.424363,37.800414
3,3,1,-122.426995,37.800873
4,3,2,-122.438738,37.771541


没有归一化。

In [27]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)

KeyboardInterrupt: 

In [ ]:
from collections import OrderedDict

data_dict_new = OrderedDict(sorted(data_dict.items()))
print(data_dict_new)

In [ ]:
result_dataframe = pd.DataFrame({
    "Id": test_data["Id"]
})
for key,value in data_dict_new.items():
    result_dataframe[key] = 0
count = 0
for item in predictions:
    for key,value in data_dict.items():
        if(value == item):
            result_dataframe[key][count] = 1
    count+=1
result_dataframe.to_csv("submission_knn.csv", index=False) 

In [ ]:
from sklearn.linear_model import LogisticRegression
lgr = LogisticRegression()
lgr.fit(X_train, y_train)
predictions = knn.predict(X_test)

#print(type(predictions))
result_dataframe = pd.DataFrame({
    "Id": test_data["Id"]
})
for key,value in data_dict_new.items():
    result_dataframe[key] = 0
count = 0
for item in predictions:
    for key,value in data_dict.items():
        if(value == item):
            result_dataframe[key][count] = 1
    count+=1
result_dataframe.to_csv("submission_logistic.csv", index=False) 

In [ ]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
log.fit(X_train, y_train)
predictions = log.predict(X_test)

for key,value in data_dict_new.items():
    result_dataframe[key] = 0
count = 0
for item in predictions:
    for key,value in data_dict.items():
        if(value == item):
            result_dataframe[key][count] = 1
    count+=1
result_dataframe.to_csv("submission_logistic.csv", index=False) 